<a href="https://colab.research.google.com/github/joaquinhojman/XGBoost/blob/main/XGBoost_with_RandomSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss
import pandas as pd
import datetime as dt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from urllib.request import urlretrieve


from sklearn.model_selection import train_test_split

In [35]:
url_train = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_train.csv'
url_test = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test.csv'

data_train = pd.read_csv(url_train) #se usa para entrenar el modelo
data_test = pd.read_csv(url_test) #se usa para predecir

In [36]:
#Separo el data frame de train en uno para entrenar y uno para probar.
model_train, model_test = train_test_split(data_train, test_size=.2, random_state=1999)


In [38]:
# Supongo que el DataFrame con el que trabajo se llama 'data_train'
# 'x' son los datos (todas las columnas menos el target), 'y' es el target
x, y = data_train.iloc[:, :-1], data_train.iloc[:, -1]

# 'x' son los datos --> x_train son los datos con los que entrenar
#                   --> x_test son los datos con los que probar
# 'y' es el target  --> y_train son las soluciones con las que comparar lo entrenado con x_train
#                   --> y_test son las soluciones con las que comparar lo probado con x_test

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1999)

#Esto lo hizo chechu y esta bien, no cambiar.

In [39]:
 xg_reg = xgb.XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 6 , alpha = 10, n_estimators = 10, eta=0.2,nthread=2,objective='binary:logistic',booster='gbtree')
 xg_reg.fit(x_train, y_train)
 preds = xg_reg.predict(x_test)



In [40]:
error = log_loss(model_test['Target'], preds)
error

0.43833679761235655

In [54]:
from sklearn.model_selection import RandomizedSearchCV


loss = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
penalty = ['l1', 'l2', 'elasticnet']
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive']
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
eta0 = [1, 10, 100]

param_distributions = dict(loss=loss,
                           penalty=penalty,
                           alpha=alpha,
                           #learning_rate=learning_rate,
                           class_weight=class_weight,
                           eta0=eta0)

random = RandomizedSearchCV(estimator=xg_reg,
                            param_distributions=param_distributions,
                            scoring='roc_auc',
                            verbose=1, n_jobs=-1,
                            n_iter=1000)
random_result = random.fit(x_train, y_train)

TypeError: ignored

In [57]:
pred_tunRS = random_result.predict(x_test)

error = log_loss(y_test, pred_tunRS)
error

0.43219808435471835

In [58]:
# Sacamos el y_test_de_practica_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

stage_test_de_practica = y_test.to_frame()

oportunidades_test_de_practica['Target'] = stage_test_de_practica
y_test_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
y_test_agrupado_por_opp = y_test_agrupado_por_opp.reset_index()

In [59]:
# Sacamos el preds_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

oportunidades_test_de_practica['Target'] = pred_tunRS

preds_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_agrupado_por_opp = preds_agrupado_por_opp.reset_index()

In [60]:
# Calculamos el error con groupby hecho
error = log_loss(y_test_agrupado_por_opp['Target'], np.nan_to_num(preds_agrupado_por_opp['Target'].to_numpy()))
error

0.44845618459199593

In [61]:
preds_testRSDT = random_result.predict(data_test)
preds_testRSDT

array([0.58646226, 0.70122695, 0.6474215 , ..., 0.47620726, 0.4106129 ,
       0.3124363 ], dtype=float32)

In [63]:
# Sacamos el preds_test_agrupado_por_opp
url_test = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test.csv'
data_test = pd.read_csv(url_test) #se usa para predecir

preds_testRSDT = random_result.predict(data_test)

oportunidades_testGV = data_test['Opportunity_ID'].to_frame()

oportunidades_testGV['Target'] = preds_testRSDT

preds_test_agrupado_por_oppGV = oportunidades_testGV.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_test_agrupado_por_oppGV = preds_test_agrupado_por_oppGV.reset_index()

preds_test_agrupado_por_oppGV

,Opportunity_ID,Target
0,10689,0.584612
1,10690,0.567522
2,10691,0.536654
3,10692,0.484819
4,10693,0.712068
...,...,...
1562,12364,0.675117
1563,12365,0.476207
1564,12366,0.410613
1565,12367,0.396612


In [ ]:
from google.colab import files

preds_test_agrupado_por_oppGV.to_csv('preds_testgv.csv', index=False, header=True)
files.download('preds_testgv.csv')